# credit Doug Hart: https://github.com/dougtheeconomist

## First make sure proper necessary installations are made

brew update <br>
brew install freetds

In [1]:
!pip install pymssql

## Then run imports and test your connection:

In [2]:
import pandas as pd
import pymssql

import sys
sys.path.append('../../')
from src.pySQL_funcs import pretty_query

with open('../../src/pw2') as pw_file:
    server, user, pw, database = pw_file.readline().split(',')

In [3]:
conn = pymssql.connect(host=server,user=user,password=pw,database=database)
cur = conn.cursor()

In [4]:
def pretty_query(cur, query):
    """
    Function to Pandas-prettify for Pythonic SQL Server queries
    *NOTE* It is recommended to alias any aggregate calculation columns
           because pymssql doesn't seem to auto generate one (returns an empty string).
    """
    cur.execute(query)
    data = cur.fetchall()
    headers = [col[0] for col in cur.description]
    out = pd.DataFrame(data=data, columns=headers)
    return out

### Run the following querry to make sure that eveything works

In [5]:
query = """
SELECT MAX(year) FROM guest.players;
"""
pretty_query(cur, query)

,
0,2019


## Questions to address with this data:

#### Question 1 
Write a query to return the 5 teams who have averaged the highest number of penalties in the last 4 seasons (2016 through 2019) Your querry should return the team id number, team name(city) and the team's average penalties over this period. <br>
[Solution](#Q1)

In [7]:
query = """
Your code here;
"""
pretty_query(cur, query)

#### Question 2 
Next we want to know if the Seahawks started to draw more penalties once Pete Carroll took over as head coach in 2010?
There are several ways to do this. Your querry should return the average number of penalties per season during the Carroll era as well as the average in years prior to this under a different head coach.<br>
[Solution](#Q2)

In [8]:
query = """
Your code here;
"""
pretty_query(cur, query)

### Question 3
In order to really see if the Seahawks are being unfairly targeted for penalties we want to look at a player who has played on a couple of different teams.

To do this we'll look at Richard Sherman, who was let go by Seattle after 2017 and picked up by San Francisco. 

#### part a
Write a query that returns Sherman's average flags per season as a Seahawk and as a 49er. Are the averages different?<br>

#### part b
Write a query for the same metric, but with all former Seahawks players
[Solution](#Q3)

In [9]:
query = """
Your code here;
"""
pretty_query(cur, query)

# Solutions

# Q1

In [10]:
query = """
SELECT TOP 5 team_id, team_city, AVG(against_count) AS penalty_average
  FROM guest.teams
  WHERE year > 2015
GROUP BY team_city, team_id
ORDER BY penalty_average DESC;
"""
pretty_query(cur, query)

,team_id,team_city,penalty_average
0,24,Oakland,124
1,16,Jacksonville,123
2,29,Seattle,121
3,17,Kansas City,117
4,11,Denver,116


# Q2

In [11]:
query = """
SELECT coach_name, AVG(against_count)
  FROM guest.teams
WHERE team_city='Seattle'
GROUP BY coach_name;
"""
pretty_query(cur, query)

,coach_name,
0,Jim Mora,95
1,Pete Carroll,120


# Q3

In [12]:
#A
query = """
SELECT team, AVG(per_game)
  FROM guest.players
 WHERE player_name='E.Thomas'
 GROUP BY team;
"""
pretty_query(cur, query)

,team,
0,Baltimore,0.190000
1,Seattle,0.215000


In [13]:
#B
query = """
WITH seattle_players AS
(SELECT player_name, AVG(per_game) as seattle_pen
  FROM guest.players
WHERE team='Seattle'
GROUP BY player_name)

SELECT t.player_name, seattle_pen, not_seattle
  FROM
    (SELECT player_name, AVG(per_game) AS not_seattle
     FROM guest.players 
    WHERE player_name in (SELECT player_name FROM seattle_players)
       AND team != 'Seattle'
    GROUP BY player_name) t
 JOIN seattle_players sp
   ON t.player_name=sp.player_name
;
"""
pretty_query(cur, query)

,player_name,seattle_pen,not_seattle
0,A.Curry,0.355000,0.310000
1,A.Woods,0.190000,0.205000
2,B.Browner,0.530000,1.050000
3,B.Giacomini,0.476666,0.345000
4,B.Irvin,0.270000,0.310000
5,B.Maxwell,0.340000,0.485000
6,B.Sowell,0.390000,0.310000
7,C.Avril,0.267500,0.426666
8,C.Clemons,0.290000,0.280000
9,C.Marsh,0.390000,0.190000


In [17]:
#B
query = """
WITH seattle_players AS
(SELECT player_name, AVG(per_game) as seattle_pen
  FROM guest.players
WHERE team='Seattle'
GROUP BY player_name)

SELECT AVG(seattle_pen) AS seattle_penalties, AVG(not_seattle) AS notsea_penalties
  FROM
    (SELECT player_name, AVG(per_game) AS not_seattle
     FROM guest.players 
    WHERE player_name in (SELECT player_name FROM seattle_players)
       AND team != 'Seattle'
    GROUP BY player_name) t
 JOIN seattle_players sp
   ON t.player_name=sp.player_name
;
"""
pretty_query(cur, query)

,seattle_penalties,notsea_penalties
0,0.319074,0.342570
